* https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store

In [ ]:
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
# %time df = pd.read_csv("data/2019-Oct.csv")

In [ ]:
from glob import glob

glob("data/*.csv")

In [ ]:
chunk_size = 1e7
chunk_iter = pd.read_csv("data/2019-Nov.csv", chunksize=chunk_size)

In [ ]:
# chunk_iter.read()

In [ ]:
def downcast(df_chunk):
    for col in df_chunk.columns:
        dtypes_name = df_chunk[col].dtypes.name
        if dtypes_name.startswith("float"):
            df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="float")
        elif dtypes_name.startswith("int"):
            # 최솟값을 구해서 음수가 있을 때는 integer
            # 음수가 없을 때는 unsigned
            if df_chunk[col].min() < 0 :
                df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="integer")
            else:
                df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="unsigned")
        # 문자일 때는 category 로 변경해 줍니다.
        # 카디널리티가 높거나 텍스트 데이터에는 적합하지 않을 수 있습니다.
        elif dtypes_name.startswith("object"):
                df_chunk[col] = df_chunk[col].astype("category")
    return df_chunk

In [ ]:
start = time.time()
row_count = 0
chunk_list = [] 
for chunk in tqdm(chunk_iter):
    #print(chunk.shape)
    row_count = row_count + chunk.shape[0]
    df_chunk = downcast(chunk)
    # chunk_list.append(df_chunk)
    df_chunk.to_parquet(f"data/2019-Nov-{df_chunk.index[0]}-{df_chunk.index[-1]}.gzip", index=False)
print(row_count)
end = time.time()
print(f"{end-start:.0f}초")

In [ ]:
gzip_list = glob("data/2019-Nov*.gzip")
gzip_list

In [ ]:
%time df_parquet_list = [pd.read_parquet(gzip_file_name) for gzip_file_name in gzip_list]
len(df_parquet_list)

In [ ]:
%time df = pd.concat(df_parquet_list, ignore_index=True)
df.shape

In [ ]:
df.info()

## dask

In [ ]:
import dask.dataframe as dd

# CSV 파일 불러오기
dd_chunk = dd.read_csv('data/2019-Oct.csv', assume_missing=True, blocksize=chunk_size)

In [ ]:
dd_chunk.shape

In [ ]:
dd_csv = dd_chunk.compute()